In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelea

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-26 21:39:21--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.79MB/s    in 0.2s    

2021-06-26 21:39:21 (5.79 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge_2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
# this wasn't working for someone that messaged in the slack channel. Might have something to do with the file type. Look up tsv.gz
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [6]:
print(f'{df.count():,}')

4,880,466


In [7]:
# filter df to retrieve all rows where total_votes is greater than 20
filtered_reviews = df.filter(df.total_votes > 20)
filtered_reviews.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47272965|R3FB6BERWPEIJP|B0029OXHVY|     236056882|Wusthof Classic I...|         Kitchen|          4|           40|         43|   N|                Y|Great knife but w...|This knife has be...| 2015-08-31|
|         US|   17776853|R1D4Z38STRDQXK|B00IRW15ES|     936830997|Wabash Valley Far...|         Kitchen|          5|    

In [8]:
# create a new column where helpful votes are at least 50% of the votes
from pyspark.sql.functions import col

filtered_reviews = filtered_reviews.withColumn("helpful_reviews", col("helpful_votes") / col("total_votes"))
filtered_reviews.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_reviews|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   47272965|R3FB6BERWPEIJP|B0029OXHVY|     236056882|Wusthof Classic I...|         Kitchen|          4|           40|         43|   N|                Y|Great knife but w...|This knife has be...| 2015-08-31|0.9302325581395349|
|         US|   17776853|R1D4Z38STRDQXK|B00I

In [12]:
filtered_reviews.count()

101229

In [9]:
# filter new df where helpful_reviews is greater than 50%
helpful_over_50 = filtered_reviews.filter(filtered_reviews.helpful_reviews > 0.50)
helpful_over_50.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_reviews|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   47272965|R3FB6BERWPEIJP|B0029OXHVY|     236056882|Wusthof Classic I...|         Kitchen|          4|           40|         43|   N|                Y|Great knife but w...|This knife has be...| 2015-08-31|0.9302325581395349|
|         US|   17776853|R1D4Z38STRDQXK|B00I

In [ ]:
#helpful_over_50.count()

93246

In [10]:
# filter again to see how many helpful reviews are vine reviews.
helpful_vine_reviews = helpful_over_50.filter(helpful_over_50.vine == "Y")
helpful_vine_reviews.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_reviews|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   33146712|R1Z71RW4J9IK93|B00YCP6UEO|     238165328|De'Longhi Automat...|         Kitchen|          5|           20|         22|   Y|                N|If you make cappu...|It's really hard ...| 2015-08-29|0.9090909090909091|
|         US|   16449486|R3FVB5QI11KI9Q|B00Y

In [11]:
# filter to see how many of the helpful reviews are non-vine reviews. 
helpful_non_vine_reviews = helpful_over_50.filter(helpful_over_50.vine == "N")
helpful_non_vine_reviews.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|   helpful_reviews|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   47272965|R3FB6BERWPEIJP|B0029OXHVY|     236056882|Wusthof Classic I...|         Kitchen|          4|           40|         43|   N|                Y|Great knife but w...|This knife has be...| 2015-08-31|0.9302325581395349|
|         US|   17776853|R1D4Z38STRDQXK|B00I

In [13]:
# determine the total number of reviews
total_reviews = filtered_reviews.count()
print("Total number of reviews: " + str(f'{total_reviews:,}'))

Total number of reviews: 101,229


In [14]:
# determine the total number of 5-star reviews
five_star_reviews = (filtered_reviews.filter(filtered_reviews.star_rating == 5).count())
print("Total 5-star reviews: " + str(f'{five_star_reviews:,}'))

Total 5-star reviews: 44,378


In [16]:
# determine the percetnage of 5-star reviews for both paid (vine) and unpaid
vine_5 = filtered_reviews.filter((filtered_reviews.star_rating == 5) & (filtered_reviews.vine == "Y")).count()
print("Vine 5 star reviews: " + str(f'{vine_5:,}'))

Vine 5 star reviews: 490


In [ ]:
# 5-star vine

In [17]:
# 5-star non-vine
non_vine_5 = filtered_reviews.filter((filtered_reviews.star_rating == 5) & (filtered_reviews.vine == "N")).count()
print("Non-Vine 5 star reviews: " + str(f'{non_vine_5:,}'))

Non-Vine 5 star reviews: 43,888


In [18]:
# find percentage of 5-star reviews that were vine reviews
five_star_vine_pct = vine_5 / five_star_reviews
print("Percentage of 5-star reviews provided by vine members: " + str(f'{five_star_vine_pct:.2%}'))

Percentage of 5-star reviews provided by vine members: 1.10%


In [19]:
# find percentage of 5-star reviews that were non-vine reviews
five_star_non_vine_pct = non_vine_5 / five_star_reviews
print("Percentage of 5-star reviews provided by non-vine members: " + str(f'{five_star_non_vine_pct:.2%}'))

Percentage of 5-star reviews provided by non-vine members: 98.90%
